In [ ]:
import control
import numpy as np


def myfom(S, SettlingTimeThreshold):
    # calcula o zeta necessário usando Mp
    zeta = -np.log(S['Overshoot'] /
                   100.) / np.sqrt(np.pi**2 + np.log(S['Overshoot'] / 100.)**2)

    # calcula o wn necessário usando zeta
    try:
        wn = np.pi / (S['PeakTime'] * np.sqrt(1 - zeta**2))
    except:
        wn = -np.log(SettlingTimeThreshold * np.sqrt(1 - zeta**2)) / (
            S['SettlingTime'] * zeta)
    Fn = wn * 2 * np.pi
    wd = wn * np.sqrt(1 - zeta**2)
    Fd = wd * 2 * np.pi

    FOM = {
           'zeta': zeta, 
           'wn': wn, 
           'Fn': Fn, 
           'wd': wd, 
           'Fd': Fd
    }
    
    return FOM


def mystepinfo(sys,
               t=None,
               SettlingTimeThreshold=0.02,
               RiseTimeLimits=[0.1, 0.9]):
    #     sys = control.timeresp._get_ss_simo(sys)

    t, step_out = control.step_response(sys, t)
    step_out = step_out.flatten()

    # Steady state value
    InfValue = step_out[-1]

    # RiseTime
    tr_lower_index = (np.where(step_out >= RiseTimeLimits[0] * InfValue)[0])[0]
    tr_upper_index = (np.where(step_out >= RiseTimeLimits[1] * InfValue)[0])[0]
    RiseTime = t[tr_upper_index] - t[tr_lower_index]

    # SettlingTime
    sup_margin = (1. + SettlingTimeThreshold) * InfValue
    inf_margin = (1. - SettlingTimeThreshold) * InfValue
    SettlingTime = 0
    for i in reversed(range(t.size)):
        if (step_out[i] <= inf_margin) | (step_out[i] >= sup_margin):
            if t.size > i + 1:
                i += 1
            SettlingTime = t[i]
            break

    # Peak
    PeakIndex = np.abs(step_out).argmax()
    PeakValue = step_out[PeakIndex]
    PeakTime = t[PeakIndex]
    SettlingMax = (step_out).max()
    SettlingMin = (step_out[tr_upper_index:]).min()
    UnderShoot = step_out.min()
    OverShoot = 100. * (step_out.max() - InfValue) / (InfValue - step_out[0])

    S = {
        'RiseTime': RiseTime,
        'SettlingTime': SettlingTime,
        'SettlingMin': SettlingMin,
        'SettlingMax': SettlingMax,
        'Overshoot': OverShoot,
        'Undershoot': UnderShoot,
        'Peak': PeakValue,
        'PeakTime': PeakTime,
        'SteadyStateValue': InfValue
    }

    return S